In [1]:
import pandas as pd
import csv
import time
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# train["session_type"] = train["session"].astype('str') + "_" + train["type"]
# test["session_type"] = test["session"].astype('str') + "_" + test["type"] 
# ### session_type 만들기

In [4]:
train.tail()

,session,aid,ts,type
216716091,12899776,1737908,1661723987073,clicks
216716092,12899777,384045,1661723976974,clicks
216716093,12899777,384045,1661723986800,clicks
216716094,12899778,561560,1661723983611,clicks
216716095,12899778,32070,1661723994936,clicks


In [3]:
test.head()

,session,aid,ts,type
0,12899779,59625,1661724000278,clicks
1,12899780,1142000,1661724000378,clicks
2,12899780,582732,1661724058352,clicks
3,12899780,973453,1661724109199,clicks
4,12899780,736515,1661724136868,clicks


In [3]:
test_sam = test.copy()
test_sam.aid = ' ' + test.aid.astype('str')
test_sam = test_sam.groupby(['session','type'])['aid'].sum().reset_index()
test_sam.head()

#test 를 세션과 타입 기준으로 aid 이어붙인 열 만들기

,session,type,aid
0,12899779,clicks,59625
1,12899780,clicks,1142000 582732 973453 736515 1142000
2,12899781,carts,199008
3,12899781,clicks,141736 199008 57315 194067 199008 199008 1990...
4,12899782,carts,1494780 413962 779477 562753 476063 779477 97...


In [4]:
test_ts = test.copy()
test_ts = test_ts.groupby(['session','type'])['ts'].max().reset_index()
test_ts.head()

#test 를 세션과 타입 기준으로 가장 큰 ts 열 만들기

,session,type,ts
0,12899779,clicks,1661724000278
1,12899780,clicks,1661724155248
2,12899781,carts,1661781409993
3,12899781,clicks,1662060160406
4,12899782,carts,1661803710262


In [5]:
test_sam["session_type"] = test_sam["session"].astype('str') + "_" + test_sam["type"]
test_ts["session_type"] = test_ts["session"].astype('str') + "_" + test_ts["type"]
test_sam = pd.merge(test_sam, test_ts, how = 'left', on = 'session_type')
test_sam = test_sam.drop(["session_type", "session_y", "type_y"], axis = 1)
test_sam.columns = ("session", "type", "aid", "ts")
test_sam.head()

# 앞에서 만든 aid ts 붙이기

,session,type,aid,ts
0,12899779,clicks,59625,1661724000278
1,12899780,clicks,1142000 582732 973453 736515 1142000,1661724155248
2,12899781,carts,199008,1661781409993
3,12899781,clicks,141736 199008 57315 194067 199008 199008 1990...,1662060160406
4,12899782,carts,1494780 413962 779477 562753 476063 779477 97...,1661803710262


In [5]:
# b =[]
# for i in range (12899779,13099779):
#     b.append(max(test[test['session'] == i]['ts']))

# # 각 세션의 가장 큰 값의 ts를 a라는 리스트에 추출 
# ****append 연산 진짜진짜진짜 오래걸림****

# sess1 = pd.DataFrame()
# sess1['session'] = pd.DataFrame(list(range(12899779, 13099779)))
# sess1['ts'] = b
# sess1.head()
# # 각 세션의 가장 큰 값 ts를 데이터 프레임으로 만들기

# test_sam = pd.merge(test_sam, test_ts, how = 'left', on = 'session')
# test_sam.head()
# # 아까 만들어둔 test_sam 에 각 세션의 최대 ts 붙이기

In [6]:
train_sam = train.copy()
train_sam.aid = ' ' + train.aid.astype('str')
train_sam = train_sam.groupby(['session','type'])['aid'].sum().reset_index()

train_ts = train.copy()
train_ts = train_ts.groupby(['session','type'])['ts'].max().reset_index()

train_sam["session_type"] = train_sam["session"].astype('str') + "_" + train_sam["type"]
train_ts["session_type"] = train_ts["session"].astype('str') + "_" + train_ts["type"]
train_sam = pd.merge(train_sam, train_ts, how = 'left', on = 'session_type')
train_sam = train_sam.drop(["session_type", "session_y", "type_y"], axis = 1)
train_sam.columns = ("session", "type", "aid", "ts")
train_sam.head()

In [8]:
# sess = pd.DataFrame()
# sess['session'] = pd.DataFrame(list(range(0, 12899778)))
# type = pd.DataFrame()
# type['type'] = ('clicks','carts','orders')
# train_sample = pd.merge(sess, type, how = 'cross')

# sess = pd.DataFrame()
# sess['session'] = pd.DataFrame(list(range(0, 12899778)))
# type = pd.DataFrame()
# type['type'] = ('clicks','carts','orders')

In [40]:
sample = pd.read_csv('sample_submission.csv')
sample.head()

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524


In [41]:
sample = pd.merge(sample, test_ts, how = 'left', on = 'session_type')
sample['session'] = sample['session_type'].str.slice(0,8)
sample['type'] = sample['session_type'].str.slice(9,15)
sample['ts'].fillna(1662328772216, inplace = True)    # max(test.ts) = 1662328772216
sample = sample.drop(["session_type", "labels"], axis = 1)
sample.head()

,session_type,labels,session,type,ts
0,12899779_clicks,129004 126836 118524,12899779,clicks,1.661724e+12
1,12899779_carts,129004 126836 118524,12899779,carts,1.662329e+12
2,12899779_orders,129004 126836 118524,12899779,orders,1.662329e+12
3,12899780_clicks,129004 126836 118524,12899780,clicks,1.661724e+12
4,12899780_carts,129004 126836 118524,12899780,carts,1.662329e+12


In [39]:
max(test.ts)

1662328772216

In [45]:
target = 'aid'

features = train_sam.drop(columns=[target]).columns

X_train = train_sam[features]
y_train = train_sam[target]
X_val = test_sam[features]
y_val = test_sam[target]
X_test = sample[features]

In [46]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

enc = OrdinalEncoder()
imp_mean = SimpleImputer()
model_dt = DecisionTreeClassifier(random_state=1, criterion="entropy")

X_train_encoded = enc.fit_transform(X_train)
X_train_imputed = imp_mean.fit_transform(X_train_encoded)
model_dt.fit(X_train_imputed, y_train)

X_val_encoded = enc.transform(X_val)
X_val_imputed = imp_mean.transform(X_val_encoded)

# score method: Return the mean accuracy on the given test data and labels
print("훈련 정확도", model_dt.score(X_train_imputed, y_train))
print("검증 정확도", model_dt.score(X_val_imputed, y_val))

X_test_encoded = enc.transform(X_test)
X_test_imputed = imp_mean.transform(X_test_encoded)

y_pred = model_dt.predict(X_test_imputed)

In [ ]:
c = pd.DataFrame(y_pred)
sample['labels'] = c
sample['session_type'] = sample['session'].astype('str') + "_" + sample['type']
sample = sample.drop(['ts'], axis = 1)
sample.head()

In [ ]:
sample = pd.read_csv('sample_submission.csv')
sample